# Daily User Data

In [1]:
import pandas as pd
from datetime import date
from datetime import timedelta
import datetime
import random
import numpy as np

## Data Preperation

### Load Data

In [2]:
transactions = pd.read_csv('transactions2.csv')
transactions['datetime']=pd.to_datetime(transactions['timestamp'],unit='s')
transactions.info()

/usr/local/etc/anaconda3/envs/blockchain/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3165: DtypeWarning: Columns (2,3,14,15,17,26,27,30,31,34,35) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 816983 entries, 0 to 816982
Data columns (total 37 columns):
 #   Column                     Non-Null Count   Dtype         
---  ------                     --------------   -----         
 0   amount                     594263 non-null  float64       
 1   borrowRate                 120765 non-null  float64       
 2   borrowRateMode             120765 non-null  object        
 3   onBehalfOf                 594263 non-null  object        
 4   pool                       816983 non-null  object        
 5   reserve                    809519 non-null  object        
 6   timestamp                  816983 non-null  int64         
 7   user                       816983 non-null  object        
 8   type                       816983 non-null  object        
 9   reservePriceETH            594263 non-null  float64       
 10  reservePriceUSD            594263 non-null  float64       
 11  amountETH                  594263 non-null  float64 

In [3]:
rates = pd.read_csv('rates.csv')
rates['datetime'] = pd.to_datetime(rates['datetime'])
rates.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 563425 entries, 0 to 563424
Data columns (total 6 columns):
 #   Column              Non-Null Count   Dtype         
---  ------              --------------   -----         
 0   liquidityRate       563425 non-null  float64       
 1   reserve             563425 non-null  object        
 2   stableBorrowRate    563425 non-null  float64       
 3   timestamp           563425 non-null  int64         
 4   variableBorrowRate  563425 non-null  float64       
 5   datetime            563425 non-null  datetime64[ns]
dtypes: datetime64[ns](1), float64(3), int64(1), object(1)
memory usage: 25.8+ MB


In [4]:
prices = pd.read_csv('prices.csv')
prices['datetime'] = pd.to_datetime(prices['datetime'])
prices.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 453795 entries, 0 to 453794
Data columns (total 4 columns):
 #   Column     Non-Null Count   Dtype         
---  ------     --------------   -----         
 0   price      453795 non-null  float64       
 1   timestamp  453795 non-null  int64         
 2   symbol     453795 non-null  object        
 3   datetime   453795 non-null  datetime64[ns]
dtypes: datetime64[ns](1), float64(1), int64(1), object(1)
memory usage: 13.8+ MB


In [5]:
tokens = pd.read_csv('tokens.csv')
tokens.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 52 entries, 0 to 51
Data columns (total 9 columns):
 #   Column                       Non-Null Count  Dtype  
---  ------                       --------------  -----  
 0   borrowingEnabled             52 non-null     bool   
 1   decimals                     52 non-null     int64  
 2   id                           52 non-null     object 
 3   pool                         52 non-null     object 
 4   reserveLiquidationBonus      52 non-null     float64
 5   reserveLiquidationThreshold  52 non-null     float64
 6   stableBorrowRateEnabled      52 non-null     bool   
 7   symbol                       52 non-null     object 
 8   usageAsCollateralEnabled     52 non-null     bool   
dtypes: bool(3), float64(2), int64(1), object(3)
memory usage: 2.7+ KB


## Main Algorithm

### Gets User's Transactions

In [6]:
def getTransactions(user):
    #get transactions involving user
    trans = transactions[((transactions['type']=='liquidation') & (transactions['user']==user))|((transactions['type']=='swap') & (transactions['user']==user))|(transactions['onBehalfOf']==user)].sort_values('timestamp')
    if len(trans)==0:
        return pd.DataFrame()
    #set up data structures to keep track of user balances
    assets = dict()
    liabilities = dict()
    borrows = dict()
    
    #get user balances for date of their first transaction up until todays date
    dates = pd.date_range(trans['datetime'].min(),trans['datetime'].max()+timedelta(days=1),freq='d')
    days=[]
    for date in dates:
        #get transactions that occured on current date
        day_trans = trans[(trans['datetime'].dt.year==date.year)&(trans['datetime'].dt.month==date.month)&(trans['datetime'].dt.day==date.day)].sort_values('timestamp')
        #go through each transactions and calculate balances
        day_trans.apply(lambda x: doTransactions(x, user, assets, liabilities, borrows),axis=1)
        calcInterest(date, rates, assets, liabilities, borrows)
    
        #compile data for day
        day = dict()
        day['datetime'] = datetime.date(date.year,date.month,date.day)
        #get price of USDT for USD price calculations
        USDT_price = getPrice(date, 'USDT')
        #add asset data
        for asset in assets.keys():
            if str(asset)=='nan':
                continue
            #get amount of each asset deposited
            day[str(asset)+'_deposited']=assets[asset]
            #get price of asset
            day[str(asset)+'_price'] = getPrice(date, asset)
            #get amount of asset deposited in ETH
            day[str(asset)+'_ETH_deposited'] = day[str(asset)+'_deposited']*day[str(asset)+'_price']
            #get amount of asset deposited in USD
            day[str(asset)+'_USD_deposited'] = day[str(asset)+'_ETH_deposited']/USDT_price
        #sum all assets
        day['sum_ETH_deposited'] = sumAssets(day)
        day['sum_USD_deposited'] = day['sum_ETH_deposited']/USDT_price
        
        #add liability data
        for liability in liabilities.keys():
            if str(liability)=='nan':
                continue
            #get amount of each liability borrowed
            day[str(liability)+'_borrowed']=liabilities[liability]  
            #get price of each liability borrowed
            day[str(liability)+'_price'] = getPrice(date, liability)
            #get amount of libability borrowed in ETH
            day[str(liability)+'_ETH_borrowed'] = day[str(liability)+'_borrowed']*day[str(liability)+'_price']
            #get amount of liabiltiy borrowed in USD
            day[str(liability)+'_USD_borrowed'] = day[str(liability)+'_ETH_borrowed']/USDT_price
        #sum all liabilties    
        day['sum_ETH_borrowed'] = sumLiabilities(day)
        day['sum_USD_borrowed'] = day['sum_ETH_borrowed']/USDT_price
        #get health for user
        if day['sum_ETH_borrowed']>0:
            day['health_factor'] = day['sum_ETH_deposited']/day['sum_ETH_borrowed']
        else: 
            day['health_factor'] = np.nan
        
        days.append(day)
    
    return pd.DataFrame(days)

### Borrow Class

In [7]:
class Borrow:
    #keep track of variables beloning to borrow
    def __init__(self, reserve, mode, rate, amount):
        self.reserve = reserve 
        self.mode = mode
        self.rate = rate
        self.amount = amount
    
    def Exists(self, reserve, mode, rate):
        if mode == 'Variable':
            return self.mode == 'Variable' and self.reserve == reserve
        else:
            return self.mode == 'Stable' and self.reserve == reserve and self.rate == rate
        
    def __lt__(self, other):
        return self.rate < other.rate
    
    def __repr__(self):
        return "Reserve: {}; Mode: {}; Rate: {}; Amount {}".format(self.reserve,self.mode,self.rate,self.amount)

### Executes Transactions

In [8]:
def doTransactions(transaction, user, assets, liabilities, borrows):
    #for deposit
    if transaction['type']=='deposit':
        #token exists in user's assets
        if transaction['reserve'] in assets.keys():
            assets[transaction['reserve']]+=transaction['amount']
        #token does not exist in user's assets
        else:
            assets[transaction['reserve']]=transaction['amount']
                
    #for redeem
    elif transaction['type']=='redeem':
        #token exists in user's assets
        if transaction['reserve'] in assets.keys():
            assets[transaction['reserve']]-=transaction['amount']
        #token does not exist in user's assets
        else:
            failures.add((user,'Redeem'))
            #print('Redeem Failure:',user)
            assets[transaction['reserve']]=-transaction['amount']
            
    #for borrow
    elif transaction['type']=='borrow':
        #print('Borrow:')
        #print(borrows)
        #token exists in user's liabilities
        if transaction['reserve'] in liabilities.keys():
            liabilities[transaction['reserve']]+=transaction['amount']
            
            #find specific borrow
            found = -1
            for i in range(len(borrows[transaction['reserve']])):
                # if found, update loan
                if borrows[transaction['reserve']][i].Exists(transaction['reserve'],transaction['borrowRateMode'],transaction['borrowRate']):
                    found = i
                    break
                    
            #if loan found, update amount 
            if i!=-1:
                borrows[transaction['reserve']][i].amount+=transaction['amount']
            #add new borrow if loan not found
            else:
                borrows[transaction['reserve']][i].append(Borrow(transaction['reserve'],transaction['borrowRateMode'],transaction['borrowRate'],transaction['amount']))   
                
        #token does not exist in user's liabilities
        else:
            liabilities[transaction['reserve']]=transaction['amount']
            borrows[transaction['reserve']] = [Borrow(transaction['reserve'], transaction['borrowRateMode'], transaction['borrowRate'], transaction['amount'])]
        #print(borrows)
        #print()
    #for repay
    elif transaction['type']=='repay':
        #print('Repay:')
        #print(borrows)
        #token exists in user's liabilities
        if transaction['reserve'] in liabilities.keys():
            liabilities[transaction['reserve']]-=transaction['amount']
            
            #sort borrows by inerest amount, and repay loans with largest rate
            amount = transaction['amount']
            borrows[transaction['reserve']].sort(reverse=True)
            for i in range(len(borrows[transaction['reserve']])):
                #end if no momey left to be repaid
                if amount<0:
                    break
                #repay amount is not enough to cover borrow
                if amount<=borrows[transaction['reserve']][i].amount:
                    borrows[transaction['reserve']][i].amount-=amount
                    break
                #repay amount covers borrow
                else:
                    amount-=borrows[transaction['reserve']][i].amount
                    borrows[transaction['reserve']][i].amount=0
            #print(borrows)
            #print()
            
        #token does not exist in user's liabilities
        else:
            failures.add((user,'Repay'))
            #print('Repay Failure:',user)
            liabilities[transaction['reserve']]=-transaction['amount']
            borrows[transaction['reserve']] = [Borrow(transaction['reserve'], 'Variable', 0, -transaction['amount'])]
            #print(borrows)
    #for liquidation
    elif transaction['type']=='liquidation':
        #liquidate principal
        if transaction['principalReserve'] in liabilities.keys():
            liabilities[transaction['principalReserve']]-=transaction['principalAmount']
        else:
            failures.add((user,'Liquidation Principal'))
            #print('Liquidation Principal Failure:',user)
            liabilities[transaction['principalReserve']]=-transaction['principalAmount']
            borrows[transaction['principalReserve']] = [Borrow(transaction['principalReserve'], 'Variable', 0, -transaction['principalAmount'])]
        #return collateral
        if transaction['collateralReserve'] in assets.keys():
            assets[transaction['collateralReserve']]-=transaction['collateralAmount']
        else:
            failures.add((user, 'Liquidation Collateral'))
            #print('Liquidation Collateral Failure:',user)
            assets[transaction['collateralReserve']]=-transaction['collateralAmount']
    #for swaps
    elif transaction['type']=='swap':
        #print('Swap:',transaction['borrowRateModeFrom'])
        #print(borrows)
        #variable to stable
        if transaction['reserve'] in liabilities.keys():
            if transaction['borrowRateModeFrom']=='Variable':
                for i in range(len(borrows[transaction['reserve']])):
                    #find variable loan
                    if borrows[transaction['reserve']][i].mode=='Variable' and borrows[transaction['reserve']][i].amount!=0:
                        #add stable loan and set variable loan to 0
                        borrows[transaction['reserve']].append(Borrow(transaction['reserve'],'Stable',transaction['stableBorrowRate'],borrows[transaction['reserve']][i].amount))
                        borrows[transaction['reserve']][i].amount=0
                        break
            #stable to variable
            else:
                for i in range(len(borrows[transaction['reserve']])):
                    #find stable loans
                    amount=0
                    if borrows[transaction['reserve']][i].mode=='Stable' and borrows[transaction['reserve']][i].amount!=0:
                        #sum up amount put in variable loans
                        amount+=borrows[transaction['reserve']][i].amount
                        borrows[transaction['reserve']][i].amount=0
                #add stable loan
                borrows[transaction['reserve']].append(Borrow(transaction['reserve'],'Variable',transaction['variableBorrowRate'],amount))
        else:
            failures.add((user,'Swap'))
            #print('Swap Failure:',user)
        #print(borrows)
        #print()

### Calculate Interest

In [9]:
def calcInterest(date, rates, assets, liabilities, borrows):
    #calculate interest on assets
    for asset in assets.keys():
        #get deposit rates for token for day
        rates_token = rates[rates['reserve']==asset]
        rates_time = rates_token[(rates_token['datetime'].dt.year==date.year)&(rates_token['datetime'].dt.month==date.month)&(rates_token['datetime'].dt.day==date.day)]
        deposit_rates = list(rates_time['liquidityRate'])
        deposit_times = list(rates_time['datetime'])
        #if no rates available for day, get last possible rate
        if len(deposit_rates)==0:
            rates_time = rates_token[rates_token['datetime']<=datetime.datetime(date.year,date.month,date.day,0,0,0)]
            deposit_rates = [list(rates_time.sort_values('timestamp')['liquidityRate'])[-1]]
            deposit_times = [datetime.datetime(date.year,date.month,date.day,0,0,0)]
        
        #calculate interest for each rate period
        today = datetime.datetime(date.year,date.month,date.day,0,0,0)
        for i in range(len(deposit_rates)):
            #get elapsed years
            elapsed = (deposit_times[i]-today).total_seconds()/(86400*365)
            #get interest rate
            rate = deposit_rates[i]/100
            #calculate interest
            if elapsed!=0:
                assets[asset]+=assets[asset]*(rate*elapsed)
            #update date
            today = deposit_times[i]
        #calculate interest for remaining part of day
        eod = datetime.datetime(date.year,date.month,date.day,23,59,59)
        elapsed = (eod-today).total_seconds()/(86400*365)
        if elapsed!=0:
            assets[asset]+=assets[asset]*(rate*elapsed)
        
    #calculate interest on liabilities
    for liability in liabilities.keys():
        #calculate for each borrow
        for i in range(len(borrows[liability])):
            #if stable loan
            if borrows[liability][i].mode=='Stable':
                liabilities[liability]+=liabilities[liability]*borrows[liability][i].rate/(100*365)
            #if variable loan
            else:
                #get borrow rates for token for day
                rates_token = rates[rates['reserve']==liability]
                rates_time = rates_token[(rates_token['datetime'].dt.year==date.year)&(rates_token['datetime'].dt.month==date.month)&(rates_token['datetime'].dt.day==date.day)]
                borrow_rates = list(rates_time['variableBorrowRate'])
                borrow_times = list(rates_time['datetime'])
                #if no rates, available for day, get last possible rate
                if len(borrow_rates)==0:
                    rates_time = rates_token[rates_token['datetime']<=datetime.datetime(date.year,date.month,date.day,0,0,0)]
                    borrow_rates = [list(rates_time.sort_values('timestamp')['variableBorrowRate'])[-1]]
                    borrow_times = [datetime.datetime(date.year,date.month,date.day,0,0,0)]
                    
                #calculate interest for each period
                today = datetime.datetime(date.year,date.month,date.day,0,0,0)
                for j in range(len(borrow_rates)):
                    #get elapsed years
                    elapsed = (borrow_times[j]-today).total_seconds()/(86400*365)
                    #get interest rate
                    rate = borrow_rates[j]/100
                    #calculate interest
                    if elapsed!=0:
                        liabilities[liability]+=liabilities[liability]*(rate*elapsed)
                    #update date
                    today = borrow_times[j]
                #calculate interest for remaining part of day
                eod = datetime.datetime(date.year,date.month,date.day,23,59,59)
                elapsed = (eod-today).total_seconds()/(86400*365)
                if elapsed!=0:
                    liabilities[liability]+=liabilities[liability]*(rate*elapsed)
                #update rate for loan   
                borrows[liability][i].rate = rate*100
        

### Price Fetcher

In [10]:
#returns price at end of day on a certain date
def getPrice(date, reserve):
    #if asset is WETH, then the price in ETH must be 1
    if reserve=='WETH' or reserve=='AmmWETH':
        return 1
    #get datetime at end of day
    date_time = datetime.datetime(date.year, date.month, date.day, 23, 59, 59)
    #get most recent price before this datetime
    price = list(prices[(prices['datetime']<=date_time)&(prices['symbol']==reserve)]['price'])[-1]
    return price

### Sum Assets

In [11]:
#sums all assets
def sumAssets(day):
    amount = 0
    #iterate through each balance for the day
    for key in day.keys():
        #if it is a deposit, add it to sum
        if key.find('_ETH_deposited')!=-1:
            amount+=day[key]
    return amount

### Sum Liabilities

In [12]:
#sums all liabilities
def sumLiabilities(day):
    amount = 0
    #iterate through each balance for the day
    for key in day.keys():
         #if it is a borrows, add it to sum
        if key.find('_ETH_borrowed')!=-1:
            amount+=day[key]
    return amount

### Driver

In [ ]:
#keep track of users that generate errors
failures = set()
#get set of all user ID's
all_users = set(transactions['user'].dropna().unique()).union(set(transactions['onBehalfOf'].dropna().unique()))
#compile data for every user in to one DataFrame
all_balances = pd.DataFrame()
i=0
version=1
for user in all_users:
    #collect user data
    try:
        balance = getTransactions(user)
    except Exception as e:
        print('Exception:', user)
        print(e)
        break
    balance['user'] = user
    
    all_balances = all_balances.append(balance)
    #update progress
    i+=1
    if i%100==0:
        print(str(i)+'/'+str(len(all_users)))
    
    #write dataframe for every 10,000 users
    if i%10000==0:
        all_balances.to_csv('all_balances'+str(version)+'.csv',index=False)
        all_balances = pd.DataFrame()
        version+=1
        print('Wrote version',version)

all_balances.info()

100/57519
200/57519
300/57519
400/57519
500/57519
600/57519
700/57519
800/57519
900/57519
1000/57519
1100/57519
1200/57519
1300/57519
1400/57519
1500/57519
1600/57519
1700/57519
1800/57519
1900/57519
2000/57519
2100/57519
2200/57519
2300/57519
2400/57519
2500/57519
2600/57519
2700/57519
2800/57519
2900/57519
3000/57519
3100/57519
3200/57519
3300/57519
3400/57519
3500/57519
3600/57519
3700/57519
3800/57519
3900/57519
4000/57519
4100/57519
4200/57519
4300/57519
4400/57519
4500/57519
4600/57519
4700/57519
4800/57519
4900/57519
5000/57519
5100/57519
5200/57519
5300/57519
5400/57519
5500/57519
5600/57519
5700/57519
5800/57519
5900/57519
6000/57519
6100/57519
6200/57519
6300/57519
6400/57519
6500/57519
6600/57519
6700/57519
6800/57519
6900/57519
7000/57519
7100/57519
7200/57519
7300/57519
7400/57519
7500/57519
7600/57519
7700/57519
7800/57519
7900/57519
8000/57519
8100/57519
8200/57519
8300/57519
8400/57519
8500/57519
8600/57519
8700/57519
8800/57519
8900/57519
9000/57519
9100/57519
9200/575

In [ ]:
good

In [15]:
all_balances.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 881934 entries, 0 to 95
Columns: 315 entries, datetime to PAX_USD_deposited
dtypes: float64(313), object(2)
memory usage: 2.1+ GB


In [ ]:
all_balances.to_csv('all_balances.csv',index=False)